# Mimic of the DES night summary, but for Rubin Observatory

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2027-11-10"
sim_version = "3.4"

In [ ]:
# Validate the input
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d+\.\d+$', sim_version) is not None

In [ ]:
import datetime
import os
import sys
from IPython.display import display, HTML
import pandas as pd
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh.transform
import bokeh.layouts
import sqlite3
import numpy as np
import healpy
import astropy
import colorcet
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy
import healpy as hp
import astropy.units as u
from astropy.visualization import ZScaleInterval
from astropy.time import Time
from astropy.coordinates import SkyCoord, get_body
from lsst.resources import ResourcePath

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/schedview')

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
import rubin_scheduler.site_models
import schedview.compute.astro
import schedview.compute.visits
import uranography
import schedview.plot.visitmap

from rubin_sim import maf
from rubin_scheduler.data import get_baseline
from schedview.compute.camera import LsstCameraFootprintPerimeter
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from uranography.api import Planisphere, make_zscale_linear_cmap
import schedview.plot.survey_skyproj

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

In [ ]:
visit_db_fname = f'/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db'
baseline_opsim_rp = ResourcePath(visit_db_fname)
day_obs_mjd = int(Time(day_obs).mjd)
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"
use_matplotlib = True

In [ ]:
stackers = [
    maf.HourAngleStacker(),
    maf.stackers.ObservationStartDatetime64Stacker(),
    maf.stackers.TeffStacker(),
    maf.stackers.OverheadStacker()
]
visits = schedview.collect.read_opsim(baseline_opsim_rp, constraint=f"FLOOR(observationStartMJD-0.5)={day_obs_mjd}", stackers=stackers)
visits = schedview.compute.visits.add_coords_tuple(visits)
visits['previous_filter'] = visits['filter'].shift(1)

## Introduction

This report produces plots analogous to the DES `nightsum` night summaries, but using Rubin Observatory / LSST baseline simulations for its input data.

In this report, the LSST DDF fields are taken to be analogous to the DES SN fields, while all other fields are taken to be analogous to the DES wide survey fields.

## Sun and Moon

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

Modified Julian Date (MJD) is in units of days (UTC).

Local Sidereal Time (LST) is in units of degrees.

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='coordinate')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

RA, dec, alt, and az are all in units of degrees.

## Conditions and statistics

### Numbers of exposures, and gaps between them

In [ ]:
overhead_summary = schedview.compute.visits.compute_overhead_summary(visits, night_events.loc['sun_n12_setting','MJD'], night_events.loc['sun_n12_rising','MJD'])
summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
display(HTML(summary_table))

## Histogram of gaps between exposures

In [ ]:
p1 = schedview.plot.create_overhead_histogram(visits)
p2 = schedview.plot.plot_overhead_vs_slew_distance(visits)
overhead_plots = bokeh.layouts.row([p1, p2])
bokeh.io.show(overhead_plots)

"Overhead" is the time window (in seconds) between the end of the previous exposure and the end of the current exposure, minus the exposure time.

## Long gaps between exposures

In [ ]:
visits.head()

In [ ]:
num_gaps = 10
long_gap_visits = visits.sort_values('overhead', ascending=False).query('overhead>30').loc[:, ['start_date', 'overhead', 'slewDistance', 'filter', 'previous_filter']].sort_values('observationId')
long_gap_visits

These are the longest gaps between exposures that occur on the night, ordered by duration of the gaps.

Slew time and previous and current filters are noted, because filter changes are the greatest expected contributions to overhead.

## PSF Width

In [ ]:
p = schedview.plot.plot_visit_param_vs_time(visits, 'seeingFwhmEff')
p.yaxis.axis_label = "Effective PSF FWHM (asec)"
bokeh.io.show(p)

Here, the "effective PSF FWHM" serves as a proxy for the measured PSF width in the DES `nightsum`, because it includes an estimates of all modeled contributions.

In an actual report using real data, the width of the PSF measured from the image should be plotted as well, so that differences can be easily noted.

## Instrumental contributions to the PSF

In [ ]:
visits = schedview.compute.visits.add_instrumental_fwhm(visits)
p = schedview.plot.plot_visit_param_vs_time(visits, 'inst_fwhm')
p.yaxis.axis_label = "Instrumental contribution to the FWHM (asec)"
bokeh.io.show(p)

This perplexes me; I expected the instrumental contribution in simulations to be constant.

## PSF ellipticity

No ellipticity is simulated by opsim.

## Effective exposure time

In [ ]:
p = schedview.plot.plot_visit_param_vs_time(visits, 't_eff')
p.yaxis.axis_label = 'Effecive exposure time (sec.)'
p.title = "Effective exposure time"
bokeh.io.show(p)

## Sky brightness

In [ ]:
p = schedview.plot.plot_visit_param_vs_time(visits, 'skyBrightness')
p.yaxis.axis_label = 'Sky brightness (mag/asec^2'
p.title = "Sky brightness"
bokeh.io.show(p)

## Cloud cover

When run with current opsim simulations, all simulations are either completely spoiled (infinite extinction) or clear (no extinction), and what is recorded is a fraction of the sky covered by clouds.

So, where the DES nightsum plots the extinction, what is plotted here is the recorded fraction cloud cover.

In [ ]:
p = schedview.plot.plot_visit_param_vs_time(visits, 'cloud')
p.yaxis.axis_label = 'Cloud cover'
p.title = "Cloud cover"
bokeh.io.show(p)

## Visit map

In [ ]:
vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
    visits=visits,
    night_date=day_obs_date,
    timezone=timezone,
    observatory=observatory,
)

In [ ]:
bokeh.io.show(vmap)

## Survey Progress

### Map depth accumulated so far

In [ ]:
observatory.mjd = night_events.loc['night_middle', 'MJD']
conditions = observatory.return_conditions()
previous_visits = schedview.collect.read_opsim(visit_db_fname, constraint=f"observationStartMjd < {night_events.loc['sunset', 'MJD']}")

In [ ]:
metric = maf.TeffMetric()
teff_hpix = schedview.compute.maf.compute_hpix_metric_in_bands(previous_visits, metric, nside=32)

In [ ]:
if use_matplotlib:
    fig = schedview.plot.survey_skyproj.create_hpix_visit_map_grid(visits, teff_hpix, model_observatory, night_events)
else:
    map_grid = schedview.plot.create_hpix_visit_map_grid(teff_hpix, visits, conditions)
    bokeh.io.show(map_grid)

### Map the most recent visit so far

In [ ]:
metric = maf.MaxMetric('observationStartMJD')
latest_mjd_hpix = schedview.compute.maf.compute_hpix_metric_in_bands(previous_visits, metric, nside=32)
time_since_latest_hpix = {b: night_events.loc['sunset', 'MJD'] - latest_mjd_hpix[b].filled(np.nan) for b in latest_mjd_hpix}

In [ ]:
long_limit = 30
for band in time_since_latest_hpix:
    long_hpix = time_since_latest_hpix[band] > long_limit
    time_since_latest_hpix[band][long_hpix] = np.nan

In [ ]:
if use_matplotlib:
    cmap = colorcet.cm.blues_r
    fig = schedview.plot.survey_skyproj.create_hpix_visit_map_grid(visits, time_since_latest_hpix, model_observatory, night_events, vmin=0, vmax=10, cmap=cmap)
else:
    map_grid = schedview.plot.create_hpix_visit_map_grid(
        time_since_latest_hpix,
        visits,
        conditions,
        scale_limits=[10, 0],
    )
    bokeh.io.show(map_grid)

## DDF Cadence

In [ ]:
time_window_duration = 120

In [ ]:
# offset by 0.5 to get to the right rollover for day_obs, and to make the range inclusive
ddf_start_time = Time(day_obs_mjd - time_window_duration - 0.5, format='mjd')
ddf_end_time = Time(day_obs_mjd + 0.5, format='mjd')

stackers = [
    maf.stackers.ObservationStartDatetime64Stacker(),
    maf.stackers.TeffStacker(),
    maf.stackers.DayObsISOStacker(),
]

ddf_visits = schedview.collect.opsim.read_ddf_visits(visit_db_fname, ddf_start_time, ddf_end_time, stackers=stackers)

In [ ]:
nightly_ddf = schedview.compute.visits.accum_teff_by_night(ddf_visits)

In [ ]:
cadence_plots = schedview.plot.create_cadence_plot(nightly_ddf, day_obs_mjd - time_window_duration, day_obs_mjd)
bokeh.io.show(cadence_plots)

## Table of exposures

In [ ]:
displayed_columns = ['start_date', 'fieldRA', 'fieldDec', 'filter', 'visitExposureTime', 'numExposures', 't_eff', 'skyBrightness', 'seeingFwhmEff', 'cloud', 'note']
displayed_visits_df = visits.loc[:, displayed_columns]
with pd.option_context('display.max_rows', 2000):
    display(displayed_visits_df)